<a href="https://colab.research.google.com/github/AshanCV/ML-project/blob/main/Audio_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'urbansound8k:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F500970%2F928025%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240303%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240303T184013Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2a2b16a38c14a39dc73ad5c760822f8ac5e0d02cd2dbb789bf027759861e115cd4127f243eebbc6d668f9aecfce71864bb6bf551101c1d013e7d67d44ebfbca7ecd8ec72fc87ba02f672a884f61f3c37ae0f4ef9cb8b1cce87c9ce8fb065a34c48e7aa319c18c9eafae6121fe69b00e2f0d794cb4cbdf0d66cd889b00d80eb7e8c429c5ddfda931d56c0192a191346fcd9fe9973419f0e1073955831f37909dafb70859f00400a5a06bded5e9c16f7e2d8dc37dcf2c2ad9fb81d323b50ae29ab5cb3a562b5c18d4946edf9e104ed6283aa3656c77a653b86b5ded1ee08c624ff9267e2b1d390315a75457f9ab1dae8a57ad717d8a0475ba03f7507fab244fa86'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 6026232524 bytes downloaded
Downloaded and uncompressed: urbansound8k
Data source import complete.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Importing necessary libraries

# Data preprocessing
import pandas as pd
import numpy as np
import os, librosa
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Visualization
import IPython.display as ipd
import matplotlib.pyplot as plt
import seaborn as sns

# Model
from tensorflow import keras
from keras.utils import to_categorical
from keras import layers, Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Metrics
from sklearn.metrics import confusion_matrix

# Suppressing warnings
from warnings import filterwarnings
filterwarnings('ignore')

# Exploring Data

In [ ]:
# Reading meta data
data = pd.read_csv('/kaggle/input/urbansound8k/UrbanSound8K.csv')
audioPath = '/kaggle/input/urbansound8k'

data.head()

## UrbanSound8K Dataset Overview

The UrbanSound8K dataset provides a collection of audio excerpts from various urban environments. Each audio file is uniquely identified by its name, which follows a specific format : [fsID]-[classID]-[occurrenceID]-[sliceID].wav.

- **fsID** : The Freesound ID of the recording from which this excerpt (slice) is taken.

- **start and end** : The start and end times of the audio slice in the original Freesound recording.

- **salience** : A subjective salience rating of the sound, where 1 represents foreground and 2 represents background.

- **fold** : The fold number (1-10) to which this file has been allocated during cross-validation.

- **classID** : A numeric identifier of the sound class, ranging from 0 to 9. The mapping of classID to sound classes is as follows :
  - 0 : air_conditioner
  - 1 : car_horn
  - 2 : children_playing
  - 3 : dog_bark
  - 4 : drilling
  - 5 : engine_idling
  - 6 : gun_shot
  - 7 : jackhammer
  - 8 : siren
  - 9 : street_music


- **class** : The corresponding class name, such as air_conditioner, car_horn, children_playing, etc.

This dataset is suitable for tasks related to audio classification and sound event recognition, offering a diverse collection of urban sounds for analysis and model training.

In [ ]:
# Plotting count distribution of classes
plt.figure(figsize = (10, 4))
sns.countplot(y = data['class'], palette = 'viridis')
plt.title('Distribution of Classes', fontsize = 16)
plt.xlabel('Count', fontsize = 14)
plt.ylabel('Class', fontsize = 14)
plt.show()

In [ ]:
# Displaying a random waveform and spectrogram

# Randomly selecting a row
row = data.sample(1).iloc[0]

# Constructing file path
audioFile = os.path.join(audioPath, 'fold' + str(row['fold']), row['slice_file_name'])

# Loading audio file
waveform, sampleRate = librosa.load(audioFile)

# Displaying audio
print(f"Class : {row['class']}\n")
ipd.display(ipd.Audio(waveform, rate = sampleRate))

# Displaying waveform
plt.figure(figsize = (15, 4))
plt.subplot(1, 2, 1)
plt.plot(waveform)
plt.title('Waveform', fontsize = 16)
plt.xlabel('Sample Index', fontsize = 12)
plt.ylabel('Amplitude', fontsize = 12)

# Displaying spectrogram
plt.subplot(1, 2, 2)
plt.specgram(waveform, Fs = sampleRate)
plt.title('Spectrogram', fontsize = 16)
plt.xlabel('Time (s)', fontsize = 12)
plt.ylabel('Frequency (Hz)', fontsize = 12)

plt.show()

## Audio Feature Extraction Methods

When working with audio data, there are several methods for extracting meaningful features that can be used for various tasks such as audio classification. Here are three fundamental approaches :

a) **Using MFCCs Data**
   - This method involves extracting Mel-frequency cepstral coefficients (MFCCs) from the audio files. MFCCs capture the spectral characteristics of the sound, making them a popular choice for audio feature extraction.

b) **Using Spectrogram Images**
   - An alternative approach is to generate spectrogram images of the audio using the `melspectrogram` function in Librosa. These spectrogram images can then be treated as 2D data points, similar to how images are handled in computer vision tasks.

c) **Combining Both Features**
   - For a comprehensive representation, some approaches involve combining both MFCCs and spectrogram features. While this can potentially enhance the model's performance, it often requires more computational time for reading and extracting data.

### Chosen Approach

In this project, the chosen method for audio feature extraction is : **a) Using MFCCs Data**
   - This approach is selected for its effectiveness in capturing essential audio characteristics, providing a suitable foundation for audio classification tasks. It strikes a balance between informative features and computational efficiency.

Feel free to explore other methods based on your specific use case and computational resources.

# Preprocessing Data

## Data Augmentation
To enhance the variety in the dataset and improve the model's robustness, you can perform data augmentation on the audio files. Feel free to uncomment the code below and execute it. This code applies pre-emphasis, time stretching, and adds noise to the audio waveforms.

In [ ]:
def augmentData(waveform):
    # Applying pre-emphasis
    waveform = librosa.effects.preemphasis(waveform)

    # Time stretching
    if np.random.rand() < 0.5:
        rate = np.random.uniform(0.8, 1.2)
        waveform = librosa.effects.time_stretch(waveform, rate = rate)

    # Adding noise
    waveform += 0.01 * np.random.normal(size = waveform.shape)

    return waveform

In [ ]:
# Computing Mel-frequency cepstral coefficients
def mfccExtract(file):
    # Loading audio file
    waveform, sampleRate = librosa.load(file)

    # waveform = augmentData(waveform)
    features = librosa.feature.mfcc(y = waveform, sr = sampleRate, n_mfcc = 64)
    return np.mean(features, axis = 1)

In [ ]:
# Storing mfcc features along with classes in dataframe
extractAll = []

# Iterating through each row
for index, row in tqdm(data.iterrows()):
    # Constructing file path
    audioFile = os.path.join(audioPath, 'fold' + str(row['fold']), row['slice_file_name'])

    # Extracting features and appending them
    features = mfccExtract(audioFile)
    extractAll.append([features, row['class']])

In [ ]:
featuresDf = pd.DataFrame(extractAll, columns = ['Features', 'Class'])
featuresDf.head()

In [ ]:
# Converting features into numpy array
x = np.array(featuresDf['Features'].tolist())

# Encoding classes
encoder = LabelEncoder()
y = encoder.fit_transform(featuresDf['Class'])
y = to_categorical(y, num_classes = 10)

In [ ]:
# Train-validation split
trainX, testX, trainY, testY = train_test_split(x, y, stratify = y, random_state = 0)

# Defining Model

In [ ]:
model = Sequential([
    layers.Dense(1024, activation = 'relu', input_shape = (64,)),
    layers.BatchNormalization(),

    layers.Dense(512, activation = 'relu'),
    layers.BatchNormalization(),

    layers.Dense(256, activation = 'relu'),
    layers.BatchNormalization(),

    layers.Dense(128, activation = 'relu'),
    layers.BatchNormalization(),

    layers.Dense(64, activation = 'relu'),
    layers.BatchNormalization(),

    layers.Dense(32, activation = 'relu'),
    layers.BatchNormalization(),

    layers.Dense(10, activation = 'softmax')
])

# Defining optimizer, loss function, and metrics
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
# Adding early stopping to avoid overfitting
earlyStopping = EarlyStopping(
    monitor = 'val_accuracy',
    min_delta = 5e-4,
    patience = 5,
    restore_best_weights = True
)

In [ ]:
# Adding a learning rate annealer
reduceLR = ReduceLROnPlateau(
    monitor = 'val_accuracy',
    factor = 0.5,
    patience = 3,
    min_lr = 1e-5
)

In [ ]:
# Training model
history = model.fit(
    trainX, trainY,
    validation_data = (testX, testY),
    epochs = 50,
    callbacks = [earlyStopping, reduceLR]
)

In [ ]:
historyDf = pd.DataFrame(history.history)

In [ ]:
# Plotting training and validation loss
historyDf.loc[:, ['loss', 'val_loss']].plot()

In [ ]:
# Plotting training and validation accuracy
historyDf.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
# Evaluating model
score = model.evaluate(testX, testY)[1] * 100
print(f'Validation accuracy of model : {score:.2f}%')

In [ ]:
# Plotting confusion matrix
pred = np.argmax(model.predict(testX), axis = 1)
true = np.argmax(testY, axis = 1)

matrix = confusion_matrix(true, pred)

plt.figure(figsize = (12, 6))
sns.heatmap(matrix, annot = True, cbar = False, fmt = 'd', cmap = 'Blues', xticklabels = encoder.classes_, yticklabels = encoder.classes_)
plt.title('Confusion Matrix', fontsize = 16)
plt.xlabel('Predicted Class', fontsize = 14)
plt.xticks(rotation = 30)
plt.ylabel('True Class', fontsize = 14)
plt.show()

In [ ]:
# Step 1: Load your own audio file
your_audio_file = '/content/mixkit-city-traffic-ambience-2931.wav'

# Step 2: Extract features from your audio file
waveform, sampleRate = librosa.load(your_audio_file)
features = mfccExtract(your_audio_file)

# Step 3: Reshape the features and make predictions
features = features.reshape(1, -1)  # Reshape features for model input
prediction = model.predict(features)
predicted_class = encoder.classes_[np.argmax(prediction)]

print("Predicted class:", predicted_class)
